In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Model

In [2]:
import sys
sys.path.append('/home/theskyabove/infomed/chexnet/')
# from model import *

import os
import numpy as np
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from read_data import ChestXrayDataSet
from sklearn.metrics import roc_auc_score
import densenet


N_CLASSES = 2
DATA_DIR = '/home/theskyabove/resources/chest14/images'
TRAIN_IMAGE_LIST = '/home/theskyabove/infomed/chexnet/ChestX-ray14/labels/train_list.txt'
VAL_IMAGE_LIST = '/home/theskyabove/infomed/chexnet/ChestX-ray14/labels/val_list.txt'
TEST_IMAGE_LIST = '/home/theskyabove/infomed/chexnet/ChestX-ray14/labels/test_list.txt'
BATCH_SIZE = 8


class DenseNet121(nn.Module):
    def __init__(self, out_size):
        super(DenseNet121, self).__init__()
        self.densenet121 = densenet.densenet121(pretrained=True)
        num_ftrs = self.densenet121.classifier.in_features
        self.densenet121.classifier = nn.Sequential(
            nn.Linear(num_ftrs, out_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.densenet121(x)
        return x

cudnn.benchmark = True

model = DenseNet121(N_CLASSES).cuda()
model = torch.nn.DataParallel(model).cuda()

## Data

In [5]:
normalize = transforms.Normalize([0.485, 0.456, 0.406],
                                 [0.229, 0.224, 0.225])

train_dataset = ChestXrayDataSet(data_dir=DATA_DIR,
                                 image_list_file=TRAIN_IMAGE_LIST,
                                 transform=transforms.Compose([
                                     transforms.Resize(256),
                                     transforms.TenCrop(224),
                                     transforms.Lambda(
                                         lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
                                     transforms.Lambda(
                                         lambda crops: torch.stack([normalize(crop) for crop in crops]))
                                 ]))

val_dataset = ChestXrayDataSet(data_dir=DATA_DIR,
                               image_list_file=VAL_IMAGE_LIST,
                               transform=transforms.Compose([
                                   transforms.Resize(256),
                                   transforms.TenCrop(224),
                                   transforms.Lambda(
                                       lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
                                   transforms.Lambda(
                                       lambda crops: torch.stack([normalize(crop) for crop in crops]))
                               ]))

test_dataset = ChestXrayDataSet(data_dir=DATA_DIR,
                                image_list_file=TEST_IMAGE_LIST,
                                transform=transforms.Compose([
                                    transforms.Resize(256),
                                    transforms.TenCrop(224),
                                    transforms.Lambda
                                    (lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
                                    transforms.Lambda
                                    (lambda crops: torch.stack([normalize(crop) for crop in crops]))
                                ]))

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE,
                         shuffle=True, num_workers=8, pin_memory=True)

val_loader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE,
                         shuffle=False, num_workers=8, pin_memory=True)

test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE,
                         shuffle=False, num_workers=8, pin_memory=True)

## Train

In [6]:
import logging


def get_logger(name):
    logger = logging.getLogger(name)
    handler = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s %(name)-24s %(levelname)-8s %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    logger.setLevel(logging.DEBUG)
    return logger

In [ ]:
import time
import datetime
from torch.optim import Adam
from torch.autograd import Variable
from tensorboardX import SummaryWriter

log = get_logger('class_train')

experiment_name = 'densenet_v1'
models_root = '/home/theskyabove/models/'
optimizer = Adam(model.parameters(), lr=1e-3)
global_epoch = 0
max_epochs = 100
batches_per_grad_step = int(64 / BATCH_SIZE)
max_grad_norm = 5.0
epoch_train_loss = []
epoch_val_loss = []

dt_start = datetime.datetime.now(tz=datetime.timezone.utc)
dt_slug = dt_start.strftime('%Y%m%d-%H%M%S')
state_dir = os.path.join(models_root, experiment_name)
tb_dir = os.path.join(models_root, experiment_name, 'logs', dt_slug)
writer = SummaryWriter(tb_dir)

global_step = 0
while global_epoch <= max_epochs:
    epoch_time = time.time()
    
    model.train()
    optimizer.zero_grad()
    train_loss = []
    accum_steps = 0
    for i, (x, y) in enumerate(train_loader):
        if x is None or y is None:
            continue
#         if i > 10:
#             break
                
        bs, n_crops, c, h, w = x.size()
        x = x.view(-1, c, h, w).cuda()
        x = x.cuda()
        y = y.cuda()
        
        x = Variable(x)
        y = Variable(y.view(y.size()[0],))
        out = model(x)
        out_mean = out.view(bs, n_crops, -1).mean(1)
        out = out_mean
        
        loss = nn.CrossEntropyLoss()(out, y)
        train_loss.append(float(loss))
        loss.backward()

        global_step += 1
        log.debug(f'global_step {global_step} loss={float(loss):0.4f}')

        writer.add_scalar('step/loss', float(loss), global_step)

        accum_steps += 1
        if accum_steps >= batches_per_grad_step:
            if max_grad_norm is not None:
                grad_norm = torch.nn.utils.clip_grad_norm(model.parameters(),
                                                          max_norm=max_grad_norm)
                writer.add_scalar('step/grad_norm', float(grad_norm), global_step)
            optimizer.step()
            optimizer.zero_grad()
            log.debug(f'gradient step at global_step {global_step} grad_norm={float(grad_norm):0.4f}')
            accum_steps = 0

    train_loss = float(np.mean(train_loss))
    epoch_train_loss.append(train_loss)

    val_loss = []
    model.eval()
    for i, (x, y) in enumerate(val_loader):
        if x is None or y is None:
            continue
#         if i > 10:
#             break
            
        bs, n_crops, c, h, w = x.size()
        x = x.view(-1, c, h, w).cuda()
        x = x.cuda()
        y = y.cuda()
        
        x = Variable(x, volatile=True)
        y = Variable(y.view(y.size()[0],), volatile=True)
        out = model(x)
        out_mean = out.view(bs, n_crops, -1).mean(1)
        out = out_mean
        
        loss = nn.CrossEntropyLoss()(out, y)
        val_loss.append(float(loss))
    val_loss = float(np.mean(val_loss))
    epoch_val_loss.append(val_loss)

    writer.add_scalar('epoch/loss/train', train_loss, global_epoch)
    writer.add_scalar('epoch/loss/val', val_loss, global_epoch)

    global_epoch += 1

    if val_loss == min(epoch_val_loss):
        log.info(
            f'epoch {global_epoch} is considered best so far')
        
    print(f'epoch {global_epoch} took {time.time() - epoch_time:0.1f}s')

## Drafts

In [ ]:
def loc_train(experiment_name,
              model_class,
              optimizer_class,
              model_hyperparams={},
              lr_initial=1e-3,
              lr_decay_exponent=None,
              weight_decay=1e-6,
              optimizer_hyperparams_extra={},
              batch_size=1,
              batches_per_grad_step=1,
              max_grad_norm=5.0,
              max_epochs=1000,
              num_workers=2,
              cuda=None,
              models_root='/home/theskyabove/models/'):
    import sys
    import os
    sys.path.append(os.path.abspath('..'))
    import torch
    from torch.autograd import Variable
    from tensorboardX import SummaryWriter
    import datalayer
    import loc.dataset
    from framework.state import State
    from logconf import get_logger
    from model_utils import repr_layer_gradient, scalar
    import numpy as np
    import datetime
    import os
    import time

    log = get_logger('loc_train')

    if cuda is None:
        cuda = torch.cuda.is_available()

    dt_start = datetime.datetime.now(tz=datetime.timezone.utc)
    dt_slug = dt_start.strftime('%Y%m%d-%H%M%S')

    restore_checkpoint = not overfit_to_1_example
    save_epoch_checkpoint = not overfit_to_1_example

    state_dir = os.path.join(models_root, experiment_name)
    tb_dir = os.path.join(models_root, experiment_name, 'logs', dt_slug)

    writer = SummaryWriter(tb_dir)

    _model = model_class(**model_hyperparams)
    _optimizer = optimizer_class(_model.parameters(),
                                 lr=lr_initial,
                                 weight_decay=weight_decay,
                                 **optimizer_hyperparams_extra)

    state = State(state_dir, _model, _optimizer,
                  epoch_train_loss=[],
                  epoch_dev_loss=[])

    if restore_checkpoint:
        loaded_checkpoint_fn = state.restore_from_checkpoint()
    if cuda:
        state.model.cuda()

    log.info(f'model: {state.model}')
    log.info(f'optimizer: {state.optimizer}')

    if overfit_to_1_example:
        train_loader, dev_loader = loc.dataset.get_overfit_loc_dataloaders()
    else:
        train_loader, dev_loader = loc.dataset.get_loc_dataloaders(batch_size=batch_size,
                                                                   num_workers=num_workers,
                                                                   dataset_params={})

    # # for debug purpose
    # for i, (x, y) in enumerate(train_loader):
    #     if x is None or y is None:
    #         continue
    #     # if i > 0:
    #     break

    print('starting')

    try:

        while state.global_epoch <= max_epochs:

            epoch_time = time.time()

            state.model.train()
            state.optimizer.zero_grad()
            train_loss = []
            accum_steps = 0
            for i, (x, y) in enumerate(train_loader):
                if x is None or y is None:
                    continue
                # if i > 499:
                #    break
                if torch.cuda.is_available():
                    x = x.cuda()
                    y = y.cuda()
                x = Variable(x)
                y = Variable(y)
                out = state.model(x)
                loss, (score_2way, score_32way) = jaccard_similarity_multiclass_loss(out, y)
                # b_score_2way, b_score_32way = jaccard_similarity_multiclass_binarized_score(out, y)
                train_loss.append(float(loss))
                loss.backward()

                state.global_step += 1
                log.debug(
                    f'global_step {state.global_step} loss={float(loss):0.4f} \ 
                    score_2way={float(score_2way):0.4f} score_32way={float(score_32way):0.4f}')

                writer.add_scalar('step/loss', float(loss), state.global_step)
                writer.add_scalar('step/score/2way', float(score_2way), state.global_step)
                writer.add_scalar('step/score/32way', float(score_32way), state.global_step)

                accum_steps += 1
                if accum_steps >= batches_per_grad_step:
                    if max_grad_norm is not None:
                        grad_norm = torch.nn.utils.clip_grad_norm(state.model.parameters(),
                                                                  max_norm=max_grad_norm)
                        writer.add_scalar('step/grad_norm', float(grad_norm), state.global_step)
                    state.optimizer.step()
                    state.optimizer.zero_grad()
                    log.debug(f'gradient step at global_step {state.global_step} grad_norm={float(grad_norm):0.4f}')
                    accum_steps = 0

            train_loss = float(np.mean(train_loss))
            state.epoch_train_loss.append(train_loss)

            dev_loss = []
            state.model.eval()
            for x, y in dev_loader:
                if x is None or y is None:
                    continue
                if cuda:
                    x = x.cuda()
                    y = y.cuda()
                x = Variable(x, volatile=True)
                y = Variable(y, volatile=True)
                out = state.model(x)
                loss, aux = jaccard_similarity_multiclass_loss(out, y)
                dev_loss.append(scalar(loss))
            dev_loss = float(np.mean(dev_loss))
            state.epoch_dev_loss.append(dev_loss)

            writer.add_scalar('epoch/loss/train', train_loss, state.global_epoch)
            writer.add_scalar('epoch/loss/dev', dev_loss, state.global_epoch)

            state.global_epoch += 1

            if save_epoch_checkpoint:
                state.save_checkpoint()
                if dev_loss == min(state.epoch_dev_loss):
                    log.info(
                        f'epoch {state.global_epoch} is considered best so far, \
                        checkpoint to {state.BEST_CHECKPOINT_FN}')
                    os.system(
                        f'cp {os.path.join(state.state_dir, state.MAIN_CHECKPOINT_FN)} \ 
                        {os.path.join(state.state_dir, state.BEST_CHECKPOINT_FN)}')

            if lr_decay_exponent is not None:
                state.set_lr(lr_initial * lr_decay**state.global_epoch)

        log.info(f'maximum epoch acieved at epoch {state.global_epoch}')

    except KeyboardInterrupt:
        import IPython
        IPython.embed(banner1=f'\n\ntraining interrupted at step {state.global_step}, fallthrough to REPL')

In [ ]:
if __name__ == '__main__':
    import multiprocessing as mp
    mp.set_start_method('spawn')
    mp.freeze_support()
    import sys
    import os
    sys.path.append(os.path.abspath('..'))
    from torch.optim import Adam

    loc_train(
        optimizer_class=Adam,
        model_class=VNetEncoder,
        model_hyperparams=dict(layers=[64, 64, 128, 128, 256, 256, 256]),
        batch_size=1,
        batches_per_grad_step=3,
    )

## Test

In [14]:
# initialize the ground truth and output tensor
gt = torch.FloatTensor()
gt = gt.cuda()
pred = torch.FloatTensor()
pred = pred.cuda()

# switch to evaluate mode
model.eval()

for i, (inp, target) in enumerate(test_loader):
    target = target.cuda()
    gt = torch.cat((gt, target), 0)
    bs, n_crops, c, h, w = inp.size()
    input_var = torch.autograd.Variable(inp.view(-1, c, h, w).cuda(), volatile=True)
    output = model(input_var)
    output_mean = output.view(bs, n_crops, -1).mean(1)
    pred = torch.cat((pred, output_mean.data), 0)

AUROCs = compute_AUCs(gt, pred)
AUROC_avg = np.array(AUROCs).mean()
print('The average AUROC is {AUROC_avg:.3f}'.format(AUROC_avg=AUROC_avg))
for i in range(N_CLASSES):
    print('The AUROC of {} is {}'.format(CLASS_NAMES[i], AUROCs[i]))

In [27]:
save('test_gt_14', gt.cpu().numpy())
save('test_pred_14', pred.cpu().numpy())

In [91]:
y_test = load('test_gt_14.npy')
y_pred = load('test_pred_14.npy')

In [54]:
y_test = list(y_test)
y_pred = list(y_pred)

for i in range(len(y_test)):
    y_test[i] = 0 if sum(y_test[i]) == 0 else 1
    y_pred[i] = 0 if sum(y_pred[i]) == 0 else 1

y_test = asarray(y_test)
y_pred = asarray(y_pred)

In [59]:
save('test_gt_14_bin', y_test)
save('test_pred_14_bin', y_pred)

In [93]:
y_test = load('test_gt_14_bin.npy')
y_pred = load('test_pred_14_bin.npy')

In [61]:
from sklearn.metrics import *